In [17]:
import pyTigerGraph as tg
import pandas as pd
import datetime
import os 
import cfg

In [18]:
cfg.token = tg.TigerGraphConnection(host="http://1.186.48.107/", graphname="anm_real_graph").getToken(cfg.secret)[0]
conn = tg.TigerGraphConnection(host="http://1.186.48.107/", graphname="anm_real_graph", username="tigergraph",password=cfg.password,apiToken=cfg.token)

In [34]:
def to_list(row):
    try:
        return row.split(",")
    except:
        return []
def cal_epoch(val):
    try:
        dt_obj = datetime.datetime.strptime(val, '%m/%d/%Y %H:%M')  
        epoch_sec = int(dt_obj.timestamp())
        return epoch_sec
    except:
        return 0

In [38]:
import os 
path = "C:/Users/Someya.Kumari/From_LSC"
os.chdir(path) 
count =0
# def read_text_file(file_path): 
#     with open(file_path, 'r') as f: 
#         print(f.read()) 
for file in os.listdir(): 
    if file.endswith("mfst.csv"): 
        file_path = f"{path}/{file}"
        conn.upsertVertex("files_imported",file,{
            "category":"manifest",
            "date_imported":str(datetime.datetime.now())
        })
        # read_text_file(file_path) 
        # print(file_path)
        mfst_df = pd.read_csv(file_path)
        mfst_df['Status'] = mfst_df['Status'].apply(to_list)
        count+=conn.upsertVertexDataFrame(mfst_df,"bill_of_events","Manifest",{
            "weight":"Weight",
            "bil_type":"Status"
        })
        
print(count)

2645


In [39]:
path = "C:/Users/Someya.Kumari/From_LSC"
os.chdir(path) 
count_e =0
for file in os.listdir(): 
    if file.endswith("stop.csv"): 
        file_path_s = f"{path}/{file}"
        conn.upsertVertex("files_imported",file,{
            "category":"stop",
            "date_imported":str(datetime.datetime.now())
        })
        # read_text_file(file_path) 
        print(file_path_s)
        
        stop_df = pd.read_csv(file_path_s)
        stop_df['Stop Seq']=stop_df['Stop Seq'].astype(str)
        stop_df['Manifest'] = stop_df['Manifest'].astype(str)
        stop_df['Stop Location#'] = stop_df['Stop Location#'].astype(str)
        stop_df['Stop Type'] = stop_df['Stop Type'].apply(to_list)
        stop_df['id'] = stop_df['Manifest']+"-" + stop_df['Stop Seq']+"-"+stop_df['Stop Location#']
        stop_df['inter_date'] = stop_df['Scheduled Date']+ " "+stop_df['Scheduled Time']
        stop_df['epoch_sch_start'] = stop_df['inter_date'].apply(cal_epoch).astype('int64')
        stop_df['epoch_sch_end'] = stop_df['epoch_sch_start'].astype('int64') + 7200
        stop_df['must_start'] = stop_df['epoch_sch_start'].astype('int64') - 3600
        stop_df['must_end'] = stop_df['epoch_sch_end'].astype('int64') + 3600
        stop_df['load_time']=7200
        stop_df.sort_values('id')
        g_event_df = stop_df.groupby('Manifest')
        g_event_df.first()
        source = []
        target = []
        for group in g_event_df.groups:
            temp_df = g_event_df.get_group(group)
            for i in range(len(temp_df)-1):
                source.append(temp_df['id'].iloc[i])
                target.append(temp_df['id'].iloc[i+1])

        data = {
            "Source": source,
            "target": target,
            "required": True
        }
        print(stop_df)

        followby_df = pd.DataFrame(data)
        count_e+=conn.upsertVertexDataFrame(stop_df,"event","id",{
            "event_type":"Stop Type",
            "plan_start":"epoch_sch_start",
            "plan_end": "epoch_sch_end" ,
            "must_start_by":"must_start",
            "must_end_by" :"must_end",
            "load_time":"load_time"
        })
        conn.upsertEdgeDataFrame(followby_df,"event","follow_by","event","Source","target",{
            "required" : "required"
        })
        conn.upsertEdgeDataFrame(stop_df,"location","happens_at","event","Stop Location#","id",{})
        conn.upsertEdgeDataFrame(stop_df,"event","belongs_to","bill_of_events","id","Manifest",{
            "belongs_to_sequence" : "Stop Seq"
        })
print(count_e)


C:/Users/Someya.Kumari/From_LSC/2205151015_stop.csv
     Manifest Stop Seq Stop Type Stop Location# Scheduled Date Scheduled Time  \
0   106494123        1       [O]          11141     05/16/2022          00:01   
1   106494123        2       [S]         372010     05/17/2022          12:00   
2   106494124        1       [O]          11141     05/16/2022          00:01   
3   106494124        2       [S]           4226     05/18/2022          12:00   
4   106494125        1       [O]          11141     05/16/2022          00:01   
5   106494125        2       [S]           1515     05/18/2022          12:00   
6   106494126        1       [O]          11141     05/16/2022          00:01   
7   106494126        2       [S]           3972     05/18/2022          12:00   
8   106494127        1       [O]          11141     05/16/2022          00:01   
9   106494127        2       [S]         126345     05/18/2022          12:00   
10  106494128        1       [O]          11141     05/16